<a href="https://colab.research.google.com/github/renato747/Analise_de_Portefolio/blob/main/Carteira_Investimentos_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Análise de Portefólio de Instrumentos Financeiros detidos para Negociação**

# 1) Justo Valor da Carteira Consolidada

In [1]:
# Importar bibliotecas
import os
import pandas as pd
import re
import plotly.express as px
from google.colab import drive
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Acessar os ficheiros através do Google Drive
drive.mount('/content/drive')

# Caminho da pasta que contém os ficheiros
folder_path = '/content/drive/My Drive/Investimentos II/Carteira/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Função para extrair uma data no formato DD.MM.AAAA a partir do nome de um arquivo.
def extract_date_from_filename(filename):
    # Usa uma expressão regular (regex) para procurar o padrão de data no nome do arquivo.
    # O padrão (\d{2})\.(\d{2})\.(\d{4}) corresponde a:
    # - Dois dígitos (\d{2}) seguidos por um ponto (\.)
    # - Dois dígitos (\d{2}) seguidos por outro ponto (\.)
    # - Quatro dígitos (\d{4}) (ex.: "01.12.2023").
    match = re.search(r'(\d{2})\.(\d{2})\.(\d{4})', filename)

    # Se o padrão foi encontrado no nome do arquivo:
    if match:
        # Retorna a data formatada no padrão ISO (AAAA-MM-DD):
        # - group(3): o ano (AAAA)
        # - group(2): o mês (MM)
        # - group(1): o dia (DD)
        return f"{match.group(3)}-{match.group(2)}-{match.group(1)}"

    # Caso nenhuma data seja encontrada, retorna None.
    return None

# Gera uma lista de arquivos na pasta especificada (folder_path)
# que começam com 'Carteira' e terminam com '.xlsx' (arquivos Excel).
files = [f for f in os.listdir(folder_path) if f.startswith('Carteira') and f.endswith('.xlsx')]

# Exibe os nomes dos arquivos encontrados que atendem aos critérios.
print("Arquivos encontrados:", files)

Arquivos encontrados: ['Carteira 31.01.2023.xlsx', 'Carteira 28.02.2023.xlsx', 'Carteira 31.03.2023.xlsx', 'Carteira 30.04.2023.xlsx', 'Carteira 31.05.2023.xlsx', 'Carteira 30.06.2023.xlsx', 'Carteira 31.07.2023.xlsx', 'Carteira 31.08.2023.xlsx', 'Carteira 30.09.2023.xlsx', 'Carteira 31.10.2023.xlsx', 'Carteira 30.11.2023.xlsx', 'Carteira 31.12.2023.xlsx', 'Carteira 31.01.2024.xlsx', 'Carteira 29.02.2024.xlsx', 'Carteira 31.03.2024.xlsx', 'Carteira 30.04.2024.xlsx', 'Carteira 31.05.2024.xlsx', 'Carteira 30.06.2024.xlsx', 'Carteira 31.07.2024.xlsx', 'Carteira 31.08.2024.xlsx', 'Carteira 30.09.2024.xlsx']


In [4]:
# Cria um DataFrame vazio para armazenar os dados consolidados de todos os arquivos Excel.
carteira_consolidada = pd.DataFrame()

# Itera sobre os arquivos Excel encontrados.
for file in files:
    file_path = os.path.join(folder_path, file)  # Caminho completo do arquivo.

    # Lê todas as planilhas do arquivo Excel.
    sheets = pd.read_excel(file_path, sheet_name=None)  # Retorna um dicionário {nome_da_planilha: DataFrame}.

    # Itera sobre cada planilha no arquivo.
    for sheet_name, df in sheets.items():
        # Extrai a data do nome do arquivo.
        date = extract_date_from_filename(file)

        # Adiciona a coluna 'Data' ao DataFrame da planilha.
        df['Data'] = date

        # Combina o DataFrame da planilha atual com o consolidado.
        carteira_consolidada = pd.concat([carteira_consolidada, df], ignore_index=True)

# Remove linhas que estão completamente vazias em todas as colunas.
carteira_consolidada.dropna(how='all', inplace=True)

# Converte a coluna Data para datetime do pandas
carteira_consolidada['Data'] = pd.to_datetime(carteira_consolidada['Data'], format='%Y-%m-%d')

# Filtra as linhas onde a coluna 'Código de Negociação' não é nula.
# Isso remove registros sem dados relevantes nesta coluna.
carteira_consolidada = carteira_consolidada[carteira_consolidada['Código de Negociação'].notna()]

# Define as colunas que serão mantidas no DataFrame final.
colunas_para_manter = ['Código de Negociação', 'Quantidade', 'Tipo', 'Preço de Fechamento', 'Valor Atualizado', 'Data']

# Filtra o DataFrame para manter apenas as linhas onde o valor na coluna 'Tipo' é 'Cotas' ou 'Recibo'.
# Além disso, restringe o DataFrame às colunas especificadas em `colunas_para_manter`.
carteira_consolidada = carteira_consolidada.loc[carteira_consolidada['Tipo'].isin(['Cotas', 'ON', 'PN']), colunas_para_manter]

In [5]:
# Agrupa os dados de 'Valor Atualizado' por mês e calcula a soma mensal.
# 1. Define a coluna 'Data' como índice do DataFrame (necessário para o método resample).
# 2. Resample aplica o agrupamento por frequência mensal ('M').
# 3. Soma os valores da coluna 'Valor Atualizado' para cada mês.
# 4. Reseta o índice para transformar o DataFrame em formato tabular novamente.
soma_mensal = carteira_consolidada.set_index('Data').resample('M')['Valor Atualizado'].sum().reset_index()

# Renomeia as colunas do DataFrame para 'Data' e 'Saldo Mensal'.
soma_mensal.columns = ['Data', 'Saldo Mensal']

# Adiciona uma nova coluna com o mês/ano formatado no formato 'MM/YYYY'.
# Usa dt.strftime para formatar a coluna 'Data' (que está no formato datetime).
soma_mensal['Mes_Ano Formatado'] = soma_mensal['Data'].dt.strftime('%m/%Y')

# Formata os valores de 'Saldo Mensal' como strings no formato brasileiro de moeda (ex.: 1.234,56).
# 1. Usa uma função lambda para aplicar formatação numérica com duas casas decimais.
# 2. Substitui pontos e vírgulas para atender ao formato brasileiro.
soma_mensal['Saldo Mensal Formatado'] = soma_mensal['Saldo Mensal'].apply(
    lambda x: f"{x:,.2f}".replace(",", "v").replace(".", ",").replace("v", ".")
)

# Cria um gráfico de barras utilizando o plotly express.
# Configura os eixos:
# - Eixo X: 'Mes_Ano Formatado' (Mês e Ano).
# - Eixo Y: 'Saldo Mensal' (soma do valor atualizado).
# Define também o título do gráfico e os rótulos dos eixos.
fig = px.bar(
    soma_mensal,
    x='Mes_Ano Formatado',
    y='Saldo Mensal',
    labels={'Mes_Ano Formatado': 'Mês/Ano', 'Saldo Mensal': 'Saldo Mensal (R$)'},
    title='Evolução da Carteira Consolidada'
)

# Atualiza os dados de hover (dica de ferramenta) do gráfico:
# 1. Mostra o Mês/Ano (Eixo X) e o Saldo Mensal formatado (R$ com casas decimais no formato brasileiro).
# 2. Define que a informação adicional mostrada será retirada do campo customdata.
fig.update_traces(
    hovertemplate='R$ %{customdata}<extra></extra>',
    customdata=soma_mensal['Saldo Mensal Formatado']
)

# Configura o formato dos ticks no eixo Y para incluir separadores de milhares.
fig.update_layout(yaxis_tickformat=',.0f')

# Exibe o gráfico gerado.
fig.show()

In [6]:
# Criar uma nova coluna categorizada com base no valor da coluna 'Tipo'.
# - Se 'Tipo' for "ON" ou "PN", categorizamos como "Ações".
# - Se 'Tipo' for "Cotas", categorizamos como "FIIs".
carteira_consolidada['Categoria'] = carteira_consolidada['Tipo'].apply(
    lambda x: 'Ações' if x in ['ON', 'PN'] else 'FIIs' if x == 'Cotas' else 'Outros'
)

# Agrupar os dados por Data e Categoria, e calcular o saldo mensal por categoria.
soma_mensal_categoria = (
    carteira_consolidada.groupby([pd.Grouper(key='Data', freq='M'), 'Categoria'])['Valor Atualizado']
    .sum()
    .reset_index()
)

# Adicionar uma coluna formatada para o mês/ano.
soma_mensal_categoria['Mes_Ano Formatado'] = soma_mensal_categoria['Data'].dt.strftime('%m/%Y')

# Criar o gráfico de barras empilhadas usando plotly express.
fig = px.bar(
    soma_mensal_categoria,
    x='Mes_Ano Formatado',
    y='Valor Atualizado',
    color='Categoria',  # Define as categorias (Ações e FIIs) como cores empilhadas.
    labels={'Mes_Ano Formatado': 'Mês/Ano', 'Valor Atualizado': 'Saldo (R$)', 'Categoria': 'Tipo'},
    title='Carteira Consolidada de Investimentos por Categoria'
)

# Personalizar o layout e dicas de ferramenta.
fig.update_traces(
    hovertemplate='R$ %{y:,.2f}<extra></extra>'
)

fig.update_layout(
    yaxis_tickformat=',.0f',  # Formato dos ticks do eixo Y.
    barmode='stack'  # Define que as barras são empilhadas.
)

# Exibir o gráfico.
fig.show()

In [7]:
# Definindo a função 'criar_pivot' que recebe dois parâmetros:
# 'carteira_consolidada' - o DataFrame com os dados da carteira.
# 'valor' - a coluna do DataFrame a ser agregada na tabela dinâmica.
def criar_pivot(carteira, valor):
    # Retorna uma tabela dinâmica (pivot table) com base nos parâmetros fornecidos.
    # A tabela é agrupada por 'Código de Negociação' e tem uma coluna para cada data.
    # A agregação é feita pela soma dos valores especificados em 'valor'.
    # Caso algum valor esteja ausente, ele será preenchido com zero (fill_value=0).
    return carteira.pivot_table(index='Código de Negociação',    # Usando 'Código de Negociação' como índice (linhas).
                                           columns='Data',                 # Usando 'Data' como as colunas da tabela dinâmica.
                                           values=valor,                   # A coluna cujos valores serão somados (especificado pelo parâmetro 'valor').
                                           aggfunc='sum',                  # A função de agregação utilizada é a soma dos valores.
                                           fill_value=0)                   # Preenche os valores ausentes com zero.

In [8]:
# Criar as pivots para a Carteira Consolidada
pivot_quantidade_consolidada = criar_pivot(carteira_consolidada, 'Quantidade')
pivot_preco_consolidada = criar_pivot(carteira_consolidada, 'Preço de Fechamento')
pivot_carteira_consolidada = criar_pivot(carteira_consolidada, 'Valor Atualizado')

In [9]:
# Filtrando os dados para Ações
carteira_acoes = carteira_consolidada[carteira_consolidada['Categoria'] == 'Ações']

# Criar as pivots para Ações
pivot_quantidade_acoes = criar_pivot(carteira_acoes, 'Quantidade')
pivot_preco_acoes = criar_pivot(carteira_acoes, 'Preço de Fechamento')
pivot_carteira_acoes = criar_pivot(carteira_acoes, 'Valor Atualizado')

In [10]:
# Filtrando os dados para FIIs
carteira_fiis = carteira_consolidada[carteira_consolidada['Categoria'] == 'FIIs']

# Criar as pivots para FIIs
pivot_quantidade_cotas = criar_pivot(carteira_fiis, 'Quantidade')
pivot_preco_cota = criar_pivot(carteira_fiis, 'Preço de Fechamento')
pivot_carteira_fiis = criar_pivot(carteira_fiis, 'Valor Atualizado')

**Consolidado:**

carteira_consolidada = dataframe que contém a consolidação de todos os ficheiros excel

pivot_quantidade_consolidada = contém a quantidade mensal de ações e cotas

pivot_preco_consolidada = contém o justo valor mensao por ação e por cota

pivot_carteira_consolidada = contém o justo valor mensal da carteira consolidada

**Ações:**

carteira_acoes = dataframe que contém apenas investimentos em ações

pivot_quantidade_acoes = contém a quantidade mensal de ações

pivot_preco_acoes = contém o justo valor mensal por de ação

pivot_carteira_acoes = contém o justo valor mensal da carteira de ações

**FIIS:**

carteira_fiis = dataframe que contém apenas investimentos em FIIS

pivot_quantidade_fiis = contém a quantidade mensal de cotas

pivot_preco_fiis = contém o justo valor mensal por de cota

pivot_carteira_fiis = contém o justo valor mensal da carteira de fiis

# 2) Custo de Aquisição da Carteira Consolidada

In [11]:
# Caminho onde os arquivos de Compra e Venda estão armazenados
folder_path_cv = '/content/drive/My Drive/Investimentos II/Extrato/'  # Altere para o caminho correto

In [12]:
def carregar_arquivos_cv(folder_path_cv):
    """
    Carrega e consolida os dados de arquivos Excel de uma pasta específica, filtrando apenas as colunas relevantes.

    Argumentos:
    folder_path_cv (str): Caminho para a pasta contendo os arquivos Excel de entrada/saída (compra/venda).

    Retorna:
    pd.DataFrame: DataFrame consolidado com os dados relevantes de todos os arquivos encontrados.
    """
    # Listar todos os arquivos Excel na pasta especificada
    files_cv = [f for f in os.listdir(folder_path_cv) if f.endswith('.xlsx')]

    # Inicializar um DataFrame vazio para armazenar os dados consolidados
    extrato = pd.DataFrame()

    # Iterar sobre cada arquivo Excel na pasta
    for file in files_cv:
        # Obter o caminho completo do arquivo
        file_path = os.path.join(folder_path_cv, file)

        # Ler o conteúdo do arquivo Excel em um DataFrame
        df_cv = pd.read_excel(file_path)

        # Lista das colunas desejadas no DataFrame final
        colunas_para_manter_cv = [
            'Entrada/Saída',       # Indica se a operação foi de entrada (compra) ou saída (venda)
            'Movimentação',        # Indica se foi compra, venda de ativos ou recebimento de rendimentos
            'Data',                # Data da operação
            'Produto',             # Produto envolvido na operação
            'Quantidade',          # Quantidade comprada/vendida
            'Preço unitário',      # Preço por unidade do produto
            'Valor da Operação'    # Valor total da operação
        ]

        # Verificar quais colunas desejadas estão presentes no arquivo atual
        colunas_disponiveis = [col for col in colunas_para_manter_cv if col in df_cv.columns]

        # Filtrar o DataFrame para manter apenas as colunas disponíveis
        df_cv = df_cv[colunas_disponiveis]

        # Concatenar os dados do arquivo atual ao DataFrame consolidado
        extrato = pd.concat([extrato, df_cv], ignore_index=True)

    # Retornar o DataFrame consolidado com os dados de compra e venda
    return extrato

def processar_colunas(extrato):
    """
    Processa as colunas 'Produto' e 'Data' do DataFrame 'extrato', ajustando seus formatos
    e limpando linhas desnecessárias.

    Argumentos:
    extrato (pd.DataFrame): DataFrame contendo os dados de compra e venda.

    Retorna:
    pd.DataFrame: DataFrame processado com as colunas ajustadas e linhas inválidas removidas.
    """
    # Verificar e processar a coluna 'Data'
    if 'Data' in extrato.columns:
        # Converter a coluna 'Data' para o formato datetime, tratando erros como valores nulos (NaT)
        extrato['Data'] = pd.to_datetime(extrato['Data'], errors='coerce')

    # Verificar e processar a coluna 'Produto'
    if 'Produto' in extrato.columns:
        # Ajustar os valores da coluna 'Produto' para conter apenas os caracteres antes do primeiro espaço
        extrato['Produto'] = extrato['Produto'].astype(str).str.split().str[0]

    # Remover linhas que estão completamente vazias (todas as colunas NaN)
    extrato.dropna(how='all', inplace=True)

    # Retornar o DataFrame processado
    return extrato

def criar_coluna_data_mes(carteira_consolidada, extrato):
    """
    Adiciona uma coluna 'Data_Mes' nos DataFrames 'carteira_consolidada' e 'extrato',
    formatando as datas no estilo 'YYYY-MM' (ano e mês).

    Argumentos:
    carteira_consolidada (pd.DataFrame): DataFrame contendo os dados da carteira consolidada.
    extrato (pd.DataFrame): DataFrame contendo os dados de compras e vendas.

    Retorna:
    tuple: Uma tupla contendo os DataFrames 'carteira_consolidada' e 'extrato', ambos com a coluna 'Data_Mes' adicionada.
    """
    # Criar a coluna 'Data_Mes' no DataFrame 'carteira_consolidada'
    # Esta coluna extrai o ano e o mês da coluna 'Data' no formato 'YYYY-MM'
    carteira_consolidada['Data_Mes'] = carteira_consolidada['Data'].dt.strftime('%Y-%m')

    # Criar a mesma coluna 'Data_Mes' no DataFrame 'extrato'
    # Também no formato 'YYYY-MM', para facilitar comparações ou agrupamentos por período
    extrato['Data_Mes'] = extrato['Data'].dt.strftime('%Y-%m')

    # Retornar os dois DataFrames com as novas colunas
    return carteira_consolidada, extrato

def mapear_datas_para_mes(carteira_consolidada, extrato):
    """
    Ajusta as datas no DataFrame 'extrato' para corresponder às datas completas
    do DataFrame 'carteira_consolidada', utilizando a coluna 'Data_Mes' como referência.

    Argumentos:
    carteira_consolidada (pd.DataFrame): DataFrame contendo as datas completas e a coluna 'Data_Mes'.
    extrato (pd.DataFrame): DataFrame contendo a coluna 'Data_Mes' para ser ajustada.

    Retorna:
    pd.DataFrame: O DataFrame 'extrato' com as datas ajustadas na coluna 'Data_Mes'.
    """
    # Criar um dicionário para mapear os valores de 'Data_Mes' de 'carteira_consolidada' para suas datas completas.
    # - 'Data_Mes' é a chave, e a data completa (formato datetime.date) é o valor.
    mapa_datas = carteira_consolidada.set_index('Data_Mes')['Data'].dt.date.to_dict()

    # Ajustar a coluna 'Data_Mes' no DataFrame 'extrato' usando o dicionário de mapeamento.
    # - Substitui os valores de 'Data_Mes' em 'extrato' pelos valores completos de 'Data' correspondentes.
    extrato['Data_Mes'] = extrato['Data_Mes'].map(mapa_datas)

    # Retornar o DataFrame 'extrato' atualizado.
    return extrato

def classificar_produtos(extrato):
    """
    Adiciona uma coluna 'Categoria' ao DataFrame 'extrato' com base no número de caracteres da coluna 'Produto'.

    Regras:
    - Se o número de caracteres de 'Produto' for 5: Classificar como 'Ações'.
    - Se o número de caracteres de 'Produto' for 6: Classificar como 'FIIs'.
    - Caso contrário: Classificar como 'Outros'.

    Argumentos:
    extrato (pd.DataFrame): DataFrame contendo os dados de compra e venda.

    Retorna:
    pd.DataFrame: DataFrame com a nova coluna 'Categoria'.
    """
    if 'Produto' in extrato.columns:
        extrato['Categoria'] = extrato['Produto'].astype(str).apply(
            lambda x: 'Ações' if len(x) == 5 else 'FIIs' if len(x) == 6 else 'Outros'
        )
    else:
        raise ValueError("A coluna 'Produto' não está presente no DataFrame.")

    return extrato


def processar_dados(folder_path_cv, carteira_consolidada):
    """
    Processa os dados de Extrato e os integra com os dados da Carteira Consolidada.

    Argumentos:
    folder_path_cv (str): Caminho da pasta contendo os arquivos de Extrato.
    carteira_consolidada (pd.DataFrame): DataFrame contendo os dados da carteira consolidada.

    Retorna:
    pd.DataFrame: O DataFrame de Extrato processado, com as datas alinhadas ao formato da carteira consolidada.
    """
    # Etapa 1: Carregar os arquivos de Extrato
    # - Lê todos os arquivos Excel na pasta especificada e mantém apenas as colunas relevantes.
    extrato = carregar_arquivos_cv(folder_path_cv)

    # Etapa 2: Processar as colunas do DataFrame de Extrato
    # - Garante que as colunas 'Data' e 'Produto' sejam ajustadas conforme necessário.
    extrato = processar_colunas(extrato)

    # Etapa 3: Adicionar a classificação 'Ações', 'FIIs', ou 'Outros' baseada na coluna 'Produto'
    extrato = classificar_produtos(extrato)

    # Etapa 4: Criar a coluna 'Data_Mes' em ambos os DataFrames
    # - Adiciona a coluna 'Data_Mes' em 'carteira_consolidada' e 'extrato' no formato 'YYYY-MM'.
    carteira_consolidada, extrato = criar_coluna_data_mes(carteira_consolidada, extrato)

    # Etapa 5: Mapear as datas completas de 'carteira_consolidada' para 'extrato'
    # - Ajusta a coluna 'Data_Mes' em 'extrato' para corresponder às datas completas de 'carteira_consolidada'.
    extrato = mapear_datas_para_mes(carteira_consolidada, extrato)

    # Retorna o DataFrame de Extrato processado.
    return extrato

# Processar os dados
extrato = processar_dados(folder_path_cv, carteira_consolidada)

In [13]:
extrato.head()

,Entrada/Saída,Movimentação,Data,Produto,Quantidade,Preço unitário,Valor da Operação,Categoria,Data_Mes
0,Credito,Transferência - Liquidação,2023-01-23,KNCR11,5,98.17,490.85,FIIs,2023-01-31
1,Credito,Transferência - Liquidação,2023-01-23,PVBI11,1,88.03,88.03,FIIs,2023-01-31
2,Credito,Rendimento,2023-02-24,BTLG11,63,0.74,46.62,FIIs,2023-02-28
3,Credito,Transferência - Liquidação,2023-02-15,BTLG11,63,93.995,5921.68,FIIs,2023-02-28
4,Debito,Transferência - Liquidação,2023-02-14,PVBI11,1,88.1,88.1,FIIs,2023-02-28


In [14]:
# Filtrar o extrato pelos movimentos de compra de ativos
extrato_compras_vendas = extrato[extrato['Movimentação'] == 'Transferência - Liquidação']

In [15]:
def calcular_preco_medio(extrato_compras_vendas):
    """
    Calcula o preço médio por cota com base no valor da operação e quantidade.
    """
    # Garantir que as colunas estejam no formato correto
    extrato_compras_vendas['Valor da Operação'] = pd.to_numeric(extrato_compras_vendas['Valor da Operação'], errors='coerce')
    extrato_compras_vendas['Quantidade'] = pd.to_numeric(extrato_compras_vendas['Quantidade'], errors='coerce')

    # Criar a pivot table: Produto como índice, e valores calculados
    pivot_preco_medio = extrato_compras_vendas.pivot_table(
        index='Produto',  # Definir Produto como índice
        values=['Valor da Operação', 'Quantidade'],  # Usar as colunas 'Valor da Operação' e 'Quantidade'
        aggfunc='sum',  # Agregar os valores pela soma
        fill_value=0  # Preencher valores ausentes com 0
    )

    # Calcular o valor médio: Valor da Operação / Quantidade
    pivot_preco_medio['Preço Médio'] = pivot_preco_medio['Valor da Operação'] / pivot_preco_medio['Quantidade']

    # Selecionar apenas a coluna do preço médio
    pivot_preco_medio_final = pivot_preco_medio[['Preço Médio']]

    # Preencher valores NaN com o último valor conhecido da mesma linha
    pivot_preco_medio_final['Preço Médio'] = pivot_preco_medio_final['Preço Médio'].fillna(method='pad')

    # Arredondar o preço médio para 2 casas decimais
    pivot_preco_medio_final = pivot_preco_medio_final.round(2)

    return pivot_preco_medio_final

def calcular_valor_investido(pivot_quantidade_consolidada, preco_medio_por_ativo):
    """
    Calcula o valor investido baseado na quantidade de cotas e no preço médio.
    """
    # Alinhar os índices para garantir que ambos têm a mesma estrutura para a multiplicação
    pivot_quantidade_alinhado = pivot_quantidade_consolidada.reindex(preco_medio_por_ativo.index)

    # Multiplicar cada coluna de pivot_quantidade_cotas pelo Preço Médio correspondente
    preco_medio_carteira = pivot_quantidade_alinhado.mul(preco_medio_por_ativo['Preço Médio'], axis=0)

    # Arredondar os valores
    preco_medio_carteira = preco_medio_carteira.round(2)

    return preco_medio_carteira

def calcular_variacao_percentual(preco_medio_por_ativo, pivot_quantidade_consolidada, pivot_preco_consolidada):
    """
    Calcula a variação percentual entre o preço médio e o preço de mercado, e calcula o saldo de mercado e preço médio.
    """
    # Obter as últimas colunas de quantidade de cotas e preço de cota
    ultima_data_qtd = pivot_quantidade_consolidada.columns[-1]
    ultima_data_preco = pivot_preco_consolidada.columns[-1]

    # Obter as quantidades e preços das últimas colunas e transformá-los em dicionários
    ultima_quantidade = pivot_quantidade_consolidada[ultima_data_qtd].to_dict()
    ultimo_preco = pivot_preco_consolidada[ultima_data_preco].to_dict()

    # Criar as novas colunas 'Qtde' e 'Preço Mercado' em 'preco_medio_por_ativo' baseadas nos mapeamentos
    preco_medio_por_ativo['Qtde'] = preco_medio_por_ativo.index.map(ultima_quantidade)
    preco_medio_por_ativo['Preço Mercado'] = preco_medio_por_ativo.index.map(ultimo_preco)

    # Calcular a variação percentual entre 'Preço Médio' e 'Preço Mercado'
    preco_medio_por_ativo['Variação (%)'] = ((preco_medio_por_ativo['Preço Mercado'] - preco_medio_por_ativo['Preço Médio'])
                                               / preco_medio_por_ativo['Preço Médio']) * 100

    # Calcular o valor investido e o valor de mercado
    preco_medio_por_ativo['Saldo Preço Médio'] = preco_medio_por_ativo['Qtde'] * preco_medio_por_ativo['Preço Médio']
    preco_medio_por_ativo['Saldo Preço Mercado'] = preco_medio_por_ativo['Qtde'] * preco_medio_por_ativo['Preço Mercado']

    # Reordenar as colunas para que 'Qtde' seja a primeira
    cols = ['Qtde'] + [col for col in preco_medio_por_ativo.columns if col != 'Qtde']
    preco_medio_por_ativo = preco_medio_por_ativo[cols]

    # Arredondar todas as colunas numéricas para duas casas decimais
    preco_medio_por_ativo = preco_medio_por_ativo.round(2)

    # Filtrar as linhas onde 'Qtde' é superior a zero
    preco_medio_por_ativo = preco_medio_por_ativo[preco_medio_por_ativo['Qtde'] > 0]

    return preco_medio_por_ativo

def gerar_grafico(preco_medio_carteira, pivot_carteira_consolidada):
    """
    Gera o gráfico de barras empilhadas sobre o valor investido e o valor de mercado da carteira.
    """
    # Garantir que as colunas estejam no formato de data
    preco_medio_carteira.columns = pd.to_datetime(preco_medio_carteira.columns, format='%Y-%m-%d')
    pivot_carteira_consolidada.columns = pd.to_datetime(pivot_carteira_consolidada.columns, format='%Y-%m-%d')

    # Somar os valores mensais de todos os ativos
    total_valor_investido = preco_medio_carteira.sum(axis=0)  # Soma das colunas de cada mês
    total_valor_mercado = pivot_carteira_consolidada.sum(axis=0)           # Soma das colunas de cada mês

    # Calcular a mais/menos valia
    mais_menos_valia = total_valor_mercado - total_valor_investido

    # Calcular o valor patrimônio
    valor_patrimonio = total_valor_investido + mais_menos_valia

    # Criar um DataFrame consolidado para o gráfico
    df_consolidado = pd.DataFrame({
        'Data': total_valor_investido.index,
        'Valor Investido': total_valor_investido.values,
        'Mais/Menos Valia': mais_menos_valia.values,
        'Valor Patrimônio': valor_patrimonio.values
    })

    # Aplicar formatação brasileira
    def formatar_valor_brasileiro(row):
        valor_formatado = f"{row['Valor Investido']:,.2f}".replace(",", "v").replace(".", ",").replace("v", ".")
        mais_menos_valia_formatado = f"{row['Mais/Menos Valia']:,.2f}".replace(",", "v").replace(".", ",").replace("v", ".")
        valor_patrimonio_formatado = f"{row['Valor Patrimônio']:,.2f}".replace(",", "v").replace(".", ",").replace("v", ".")
        mes_ano_formatado = row['Data'].strftime('%m/%Y')
        return mes_ano_formatado, valor_formatado, mais_menos_valia_formatado, valor_patrimonio_formatado

    # Adicionar colunas formatadas ao DataFrame
    df_consolidado['Mes_Ano Formatado'], df_consolidado['Valor Investido Formatado'], df_consolidado['Mais/Menos Valia Formatado'], df_consolidado['Valor Patrimônio Formatado'] = zip(*df_consolidado.apply(formatar_valor_brasileiro, axis=1))

    # Criar o gráfico de barras empilhadas
    fig = px.bar(
        df_consolidado,
        x='Mes_Ano Formatado',
        y=['Valor Investido', 'Mais/Menos Valia'],
        labels={'variable': 'Tipo', 'value': 'Valor (R$)'},
        title='Desempenho Mensal do Portefólio de Investimento'
    )

    # Atualizar o hover template para incluir o Valor Patrimônio
    fig.update_traces(
        hovertemplate='%{x}<br>Valor Investido: R$ %{customdata[0]}<br>Mais/Menos Valia: R$ %{customdata[1]}<br>Valor Patrimônio: R$ %{customdata[2]}<extra></extra>',
        customdata=df_consolidado[['Valor Investido Formatado', 'Mais/Menos Valia Formatado', 'Valor Patrimônio Formatado']].values
    )

    # Configurar o layout do gráfico
    fig.update_layout(
        barmode='relative',  # Para barras empilhadas
        yaxis_tickformat=',.0f',  # Formatação do eixo Y
        xaxis_title='',  # Remove o título do eixo X
        xaxis=dict(
            tickmode='array',
            tickvals=df_consolidado['Mes_Ano Formatado'],
            ticktext=df_consolidado['Mes_Ano Formatado']
        )
    )

    # Exibir o gráfico
    fig.show()

In [16]:
# Calcular o preço médio por ativo com base nas compras e vendas realizadas
preco_medio_por_ativo = calcular_preco_medio(extrato_compras_vendas)

# Atualizar o DataFrame adicionando informações sobre quantidade, preço de mercado,
# variação percentual, e saldos baseados nos dados consolidados
preco_medio_por_ativo = calcular_variacao_percentual(preco_medio_por_ativo, pivot_quantidade_consolidada, pivot_preco_consolidada)

# Calcular o valor total investido na carteira com base nas quantidades consolidadas
# e nos preços médios por ativo
preco_medio_carteira = calcular_valor_investido(pivot_quantidade_consolidada, preco_medio_por_ativo)

In [17]:
# Gerar o gráfico de Desempenho Mesal do Portéfolio de Investimento
gerar_grafico(preco_medio_carteira, pivot_carteira_consolidada)

**Consolidado:**

extrato = contém todos os movimentos (compras, vendas, direito, rendimentos, etc)

preco_medio_por_ativo = contém o custo médio de aquisição e outras informações

preco_medio_carteira = contém o custo médio de aquisição da carteira


**Ações:**



**FIIS:**



In [ ]:
'''# Caminho no Google Drive
caminho_drive = '/content/drive/My Drive/extrato.xlsx'

# Salvar no Google Drive
extrato.to_excel(caminho_drive, index=False)
print(f"Arquivo salvo no Google Drive: {caminho_drive}")'''